In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math

In [11]:
class Transformer(nn.Module):
    """
    A standard encoder-decoder architecture. Acts as a base for this and other models.
    The model takes in encoder, decoder, src embed, target embed objects.
    And finally a generator object that produces predictions from the decoder.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_emebd = tgt_embed
        self.generator = generator

    def forward(self, src, src_mask, tgt, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [12]:
class Generator(nn.Module):
    """Final generation step with linear layer and softmax for output probs"""

    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        self.linear_layer = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.linear_layer(x), dim=-1)

In [44]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2

In [13]:
# Transformer consists of stacked layers of Encoder and Decoder


def generate_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Encoder stack
class EncoderStack(nn.Module):
    """
    Takes in single encoder layer. Produce stacked layers and passes input thru each layer
    """
    def __init__(self, encoder_layer, N=6):
        self.layers = generate_clones(encoder_layer, N)
        self.norm = LayerNorm(encoder_layer.size)

    def forward(self, x, src_mask):
        """
        Pass input thru each layer
        """
        for layer in self.layers:
            x = layer(x, src_mask)
        return self.norm(x)

In [26]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2

In [45]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout=0.1):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        """Apply residual connection to the output of sublayer and layer norm the result"""
        return self.norm(x + sublayer(x))

In [46]:
# A single encoder layer. It consists of two sub layers. The first is a multi-head self-attention module and 
# a fully connected position-wise feed forward network

class EncoderLayer(nn.Module):
    def __init__(self, model_size, self_attn, feed_forward, dropout=0.1):
        super(EncoderLayer, self).__init()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = model_size
        self.sublayers = generate_clones(SublayerConnection(model_size, dropout), 2)
    
    def forward(self, x, mask):
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayers[1](x, self.feed_forward)


In [47]:
class DecoderStack(nn.Module):
    def __init__(self, decoder_layer, N=6):
        super(DecoderStack, self).__init__()
        self.layers = generate_clones(decoder_layer, N)
        self.norm = LayerNorm(decoder_layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    """
    Decoder layer has 3 sublayers: self-attention, source attention and feed forward
    """
    def __init__(self, model_size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = model_size
        self.self_attn = self_attn
        # source attention module
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.dropout = nn.Dropout(dropout)
        self.sublayers = generate_clones(SublayerConnection(model_size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        # pass the data thru self attention module
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # pass the data thru source attention module
        x = self.sublayers[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        # pass the data thru feed forward module
        return self.sublayers[2](x, self.feed_forward)

In [ ]:
# Attention function
# Attention(Q, K, V) = softmax(Q @ K.T / sqrt(d_k)) @ V
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-1, -2)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
        print()#do something

    attn_scores = F.softmax(scores, dim=-1)
    if dropout is not None:
        attn_scores = dropout(attn_scores)
    result = torch.matmul(attn_scores, value)
    return result, attn_scores

In [ ]:
# Multi head Attention

class MultiHeadAttention(nn.Module):
    def __init__(self, model_size, heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        self.model_size = model_size
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        assert model_size % heads == 0, "model_size must be divisible by heads"
        self.d_k = model_size // heads
        self.linear_layers = generate_clones(nn.Linear(model_size, model_size), 4)
        self.attn = None

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        n_batches = query.size(0)

        # linearly project query, key and value in to number of heads times

        query, key, value = [
            layer(x).view(n_batches, -1, self.heads, self.d_k).transpose(1, 2) 
            for layer, x in zip(self.linear_layers, (query, key, value))
        ]

        # calculate attention using scaled dot product
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # x shape would be [n_batches, heads, seq_len, d_k]
        # concat all the heads and apply final linear layer
        x = x.transpose(1, 2).contiguous().view(n_batches, -1, self.heads * self.d_k)
        del query
        del key
        del value
        return self.linear_layers[-1](x)



In [83]:
# position wise feed forward network

class PositionwiseFeedForward(nn.Module):
    def __init__(self, model_size, d_ff, dropout=0.1):
        self.layer1 = nn.Linear(model_size, d_ff)
        self.layer2 = nn.Linear(d_ff, model_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.layer2(self.dropout(self.layer1(x).relu()))

In [ ]:
# Embeddings
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
# Positional Encoding

# PE helps model to understand the order of sequence of words. Since model 
# doesn't have any recurrence or convolution layers

class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)